In [0]:
from dotenv import load_dotenv
import os
import random
from typing import Literal, TypedDict
from langchain_core.messages import AnyMessage, HumanMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated
from databricks_langchain import ChatDatabricks
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition


from langgraph.graph import MessagesState

# load_dotenv()


import mlflow
mlflow.langchain.autolog()

class State(MessagesState):
    pass



def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    """Example:
    >>> multiply(2, 3)
    6
    """
    """Args:
    a: int
    b: int
    """
    """Returns:
    int
    """
    return a * b


llm = ChatDatabricks(endpoint = "databricks-gpt-oss-120b")

llm_with_tools = llm.bind_tools([multiply])

def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode([multiply]))


builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm", tools_condition)
builder.add_edge("tools", "tool_calling_llm")

graph = builder.compile()

result = graph.invoke({"messages": [HumanMessage("Hello,what is the weather in Tokyo? and what is 2 * 3?")]})


for i, message in enumerate(result["messages"]):
    print(f"Message {i+1} ({type(message).__name__}): {message.content}")


